In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math

In [2]:
start = time.time()
# import nodes
road_node = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\nodes.shp')
road_node = road_node.set_index('osmid')
road_nodes = road_node.to_crs(4326)
road_nodes['geometry_m'] = gpd.GeoSeries(road_nodes['geometry'], crs = 4326).to_crs(3043)
road_nodes

,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m
osmid,,,,,,,,,
103237949,4.431781e+06,503033.785157,3,-74.964440,40.036252,None,None,POINT (-74.96444 40.03625),POINT (-5689489.546 8448457.709)
103353127,4.431724e+06,502937.820011,3,-74.965565,40.035733,None,None,POINT (-74.96557 40.03573),POINT (-5689617.107 8448567.833)
103353090,4.431839e+06,503124.706820,3,-74.963374,40.036770,None,None,POINT (-74.96337 40.03677),POINT (-5689364.441 8448354.775)
103237976,4.431638e+06,503122.315306,4,-74.963403,40.034959,None,None,POINT (-74.96340 40.03496),POINT (-5689653.040 8448263.511)
103238007,4.431510e+06,503203.675365,4,-74.962450,40.033810,None,None,POINT (-74.96245 40.03381),POINT (-5689797.056 8448087.089)
...,...,...,...,...,...,...,...,...,...
9703813717,4.416438e+06,483689.901846,3,-75.190791,39.897861,None,None,POINT (-75.19079 39.89786),POINT (-5720575.973 8468970.364)
9703813743,4.416385e+06,483760.410069,3,-75.189965,39.897385,None,None,POINT (-75.18996 39.89738),POINT (-5720619.056 8468844.195)
9703813748,4.416433e+06,484049.808886,3,-75.186581,39.897820,None,None,POINT (-75.18658 39.89782),POINT (-5720414.758 8468450.854)


In [3]:
# import edges
road_edge = gpd.read_file(r'D:\Dumps\All_2km buffer\Philadelphia 2km buffer_all\edges.shp')
road_edge = road_edge.set_index(['u','v','key'])
road_edges = road_edge.to_crs(4326)
road_edges['geometry_m'] = gpd.GeoSeries(road_edges['geometry'], crs = 4326).to_crs(3043)
road_edges

#Get both ways
RE = road_edges.reset_index()
RE2 = RE[['osmid','from','to','name','length','geometry']]
RE2.columns = ['osmid','to','from','name','length','geometry']
RE2 = RE2[['osmid','from','to','name','length','geometry']]
road_connections = pd.concat([RE, RE2]).to_crs(4326)
road_connections['key'] = road_connections['from'].astype(str) + '-' + road_connections['to'].astype(str)

In [4]:
# Get the graph object
graph = ox.graph_from_place('Philadelphia, United States', network_type="all", buffer_dist = 2000)
graph = ox.speed.add_edge_speeds(graph)
graph = ox.speed.add_edge_travel_times(graph)

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [5]:
# Import greenspace and population grids
Penn_bound = gpd.read_file(r'C:\Users\bartb\Downloads\Pennsylvania\boundaries.shp')
Philly_greenspace = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\Public_greenspace.gpkg')
Philly_popgrid = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\CPoPGrid.gpkg')
Philly_bound = Penn_bound[Penn_bound['municipal1'] == 'PHILADELPHIA']

In [6]:
# Set crs to WGS84
Philly_bound = Philly_bound.to_crs(4326)
Philly_greenspace = Philly_greenspace.to_crs(4326)
Philly_popgrid = Philly_popgrid.to_crs(4326)

In [7]:
# Make sure only complete overlays progress and take their centroids
popgrid = Philly_popgrid.overlay(Philly_bound)
popgrid['centroid'] = popgrid.to_crs(4326).centroid
popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
popgrid['area'] = popgrid.area / popgrid.area.max()
popgrid = popgrid[(popgrid['area'] >= 0.99) & popgrid['PoP2015_Number'] > 0]
popgrid['grid_lon'] = popgrid['centroid_m'].x
popgrid['grid_lat'] = popgrid['centroid_m'].y
popgrid = popgrid.reset_index()
popgrid

C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\1366358376.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  popgrid['centroid'] = popgrid.to_crs(4326).centroid
C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\1366358376.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  popgrid['area'] = popgrid.area / popgrid.area.max()


,index,grid_id,PoP2015_Number,class_of_m,county,fed_aid_ur,fed_id_num,fips_area,fips_count,fips_mun_c,...,gpid,mslink,municipal,municipal1,geometry,centroid,centroid_m,area,grid_lon,grid_lat
0,6,259,113,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),0.996208,-5.675978e+06,8.459813e+06
1,7,260,135,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),0.996208,-5.675889e+06,8.459539e+06
2,13,400,137,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),0.996234,-5.676341e+06,8.459996e+06
3,16,403,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01003 40.13349, -75.01003 40.131...",POINT (-75.01115 40.13262),POINT (-5676072.561 8459176.309),0.996234,-5.676073e+06,8.459176e+06
4,17,404,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.00778 40.13349, -75.00778 40.131...",POINT (-75.00890 40.13262),POINT (-5675982.949 8458902.986),0.996234,-5.675983e+06,8.458903e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,10223,20883,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",POINT (-75.25819 39.88221),POINT (-5725754.488 8476450.010),0.999924,-5.725754e+06,8.476450e+06
3887,10225,20885,39,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",POINT (-75.25369 39.88221),POINT (-5725576.204 8475897.317),0.999924,-5.725576e+06,8.475897e+06
3888,10226,20886,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",POINT (-75.25145 39.88221),POINT (-5725487.039 8475620.982),0.999924,-5.725487e+06,8.475621e+06
3889,10275,21024,3,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",POINT (-75.26043 39.88048),POINT (-5726119.978 8476637.252),0.999949,-5.726120e+06,8.476637e+06


In [8]:
green_buffer = gpd.GeoDataFrame(geometry = Philly_greenspace.to_crs(3043).buffer(25).to_crs(4326))
Philly_greenspace['geometry_w_buffer'] = green_buffer
Philly_greenspace['geometry_w_buffer'] = gpd.GeoSeries(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['geom buffer diff'] = Philly_greenspace['geometry_w_buffer'].difference(Philly_greenspace['geometry'])

C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\3238172651.py:4: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  Philly_greenspace['geom buffer diff'] = Philly_greenspace['geometry_w_buffer'].difference(Philly_greenspace['geometry'])


In [9]:
# This function group components in itself that overlap (with the buffer set of 25 metres)
# https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
W = libpysal.weights.fuzzy_contiguity(Philly_greenspace['geometry_w_buffer'])
Philly_greenspace['components'] = W.component_labels
Philly_parks = Philly_greenspace.dissolve('components')

# Exclude parks below 0.04 ha.
Philly_parks = Philly_parks[Philly_parks.to_crs(3043).area > 400]

# Get the Philly buffer
Bound_buffer = gpd.GeoDataFrame(
    geometry = np.repeat(Philly_bound.to_crs(3043).
                         buffer(1000).to_crs(4326),len(Philly_parks)), 
    crs = 4326).reset_index().iloc[:,1]
Bound_buffer.index = Philly_parks.index
Philly_parks['bound_buffer'] = Bound_buffer
Philly_parks

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 747 disconnected components.
 There are 585 islands with ids: 11, 15, 26, 28, 45, 52, 53, 61, 65, 68, 71, 89, 90, 91, 95, 96, 97, 98, 103, 104, 105, 106, 107, 108, 110, 117, 118, 127, 133, 134, 138, 139, 148, 149, 153, 154, 155, 156, 157, 159, 163, 168, 171, 173, 174, 177, 181, 182, 185, 186, 189, 190, 191, 192, 193, 195, 197, 198, 199, 200, 201, 202, 203, 217, 221, 228, 232, 235, 239, 240, 241, 242, 245, 249, 255, 256, 261, 262, 264, 266, 267, 272, 273, 275, 276, 280, 285, 286, 288, 290, 292, 301, 303, 304, 307, 310, 313, 315, 316, 318, 320, 321, 322, 326, 327, 328, 329, 330, 331, 332, 335, 336, 337, 341, 342, 343, 344, 348, 349, 351, 355, 356, 357, 358, 359, 362, 363, 370, 371, 385, 437, 445, 449, 450, 451, 460, 461, 462, 463, 466, 467, 473, 483, 484, 485, 497, 498, 499, 503, 504, 547, 562, 564, 566, 569, 578, 579, 581, 582, 583, 5

,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,...,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,bound_buffer
components,,,,,,,,,,,,,,,,,,,,,
0,"POLYGON ((-75.09099 39.91406, -75.09099 39.914...",74457,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,4153.613200,2.281885e+05,"POLYGON ((-75.09191 39.91345, -75.09192 39.913...","POLYGON ((-75.09192 39.91345, -75.09193 39.913...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
1,"MULTIPOLYGON (((-75.02442 39.89516, -75.02516 ...",74458,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,2838.997916,5.667747e+04,"POLYGON ((-75.02347 39.89392, -75.02348 39.893...","POLYGON ((-75.02348 39.89391, -75.02350 39.893...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
2,"MULTIPOLYGON (((-75.09305 39.93204, -75.09310 ...",74458,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,2838.997916,5.667747e+04,"POLYGON ((-75.08855 39.93442, -75.08855 39.934...","POLYGON ((-75.08855 39.93442, -75.08859 39.934...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
3,"MULTIPOLYGON (((-75.06332 39.87934, -75.06355 ...",74462,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,3919.908080,1.591219e+05,"POLYGON ((-75.07984 39.88138, -75.07984 39.881...","POLYGON ((-75.07984 39.88137, -75.07983 39.881...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
4,"POLYGON ((-75.07444 39.89581, -75.07452 39.895...",74462,Fee,Fee,LOC,CNTY,Camden County,LOC,CNTY,Camden County,...,2020,None,None,None,None,3919.908080,1.591219e+05,"POLYGON ((-75.07434 39.89568, -75.07442 39.895...","POLYGON ((-75.07442 39.89565, -75.07444 39.895...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
742,"POLYGON ((-75.18593 40.13635, -75.18627 40.135...",242041,Fee,Fee,LOC,CITY,Upper Dublin Township,LOC,CITY,Upper Dublin Township,...,2020,None,None,None,None,1490.477107,6.204571e+04,"POLYGON ((-75.18593 40.13650, -75.18592 40.136...","POLYGON ((-75.18592 40.13650, -75.18590 40.136...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
743,"POLYGON ((-75.26429 40.09641, -75.26362 40.096...",244970,Fee,Fee,LOC,CITY,WHITEMARSH TOWNSHIP,LOC,CITY,WHITEMARSH TOWNSHIP,...,2020,None,None,None,None,1275.860609,8.335866e+04,"POLYGON ((-75.26442 40.09652, -75.26440 40.096...","POLYGON ((-75.26440 40.09653, -75.26438 40.096...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."
744,"POLYGON ((-75.20583 40.11865, -75.20772 40.116...",244971,Fee,Fee,LOC,CITY,WHITEMARSH TOWNSHIP,LOC,CITY,WHITEMARSH TOWNSHIP,...,2020,None,None,None,None,756.792370,2.828873e+04,"POLYGON ((-75.20571 40.11877, -75.20569 40.118...","POLYGON ((-75.20569 40.11876, -75.20568 40.118...","POLYGON ((-75.25940 39.87036, -75.26059 39.870..."


In [10]:
# Take a buffer for the parks to ensure equal measurements for edge population grids
parks_within_range = Philly_parks[Philly_parks['geometry'].intersection(Philly_parks['bound_buffer']).area > 0]
parks_within_range = parks_within_range.loc[:, ~Philly_parks.columns.isin(['bound_buffer'])]
parks_within_range = parks_within_range.reset_index()
parks_within_range['centroid'] = parks_within_range.centroid
parks_within_range['centroid_m'] = parks_within_range['centroid'].to_crs(3043)
parks_within_range['geometry_w_buffer'] = gpd.GeoSeries(parks_within_range['geometry_w_buffer'], crs = 4326)
parks_within_range['geometry_w_buffer_m'] = parks_within_range['geometry_w_buffer'].to_crs(3043)
parks_within_range = gpd.GeoDataFrame(parks_within_range, geometry = 'geom buffer diff', crs = 4326)
parks_within_range

C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\3404927833.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range = Philly_parks[Philly_parks['geometry'].intersection(Philly_parks['bound_buffer']).area > 0]
C:\Users\bartb\AppData\Local\Temp\ipykernel_18500\3404927833.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range['centroid'] = parks_within_range.centroid


,components,geometry,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,...,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry_w_buffer,geom buffer diff,centroid,centroid_m,geometry_w_buffer_m
0,5,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,...,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.25608 40.06117, -75.25606 40.061...",POINT (-75.25165 40.05653),POINT (-5697693.589 8484600.730),"POLYGON ((-5697129.449 8485382.576, -5697129.4..."
1,6,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24691 39.91683, -75.24696 39.916...","POLYGON ((-75.24696 39.91680, -75.24697 39.916...",POINT (-75.24714 39.91942),POINT (-5719375.750 8477008.358),"POLYGON ((-5719780.450 8476846.530, -5719785.6..."
2,7,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,...,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24828 39.92126, -75.24836 39.921...","POLYGON ((-75.24836 39.92121, -75.24837 39.921...",POINT (-75.25640 39.96241),POINT (-5712881.953 8480355.991),"POLYGON ((-5719127.059 8477242.926, -5719138.2..."
3,8,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,Fee,Fee,FED,NPS,NPS,FED,NPS,...,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.15037 39.96185, -75.15037 39.961...","POLYGON ((-75.15037 39.96186, -75.15038 39.961...",POINT (-75.15001 39.96195),POINT (-5708732.995 8467282.252),"POLYGON ((-5708763.625 8467321.486, -5708762.4..."
4,9,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,Fee,Fee,FED,NPS,NPS,FED,NPS,...,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"POLYGON ((-75.17280 40.03368, -75.17280 40.033...","POLYGON ((-75.17280 40.03369, -75.17280 40.033...",POINT (-75.17237 40.03387),POINT (-5698171.846 8473736.251),"POLYGON ((-5698218.752 8473779.070, -5698217.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,734,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25744 39.94930, -75.25744 39.949...","POLYGON ((-75.25744 39.94930, -75.25806 39.949...",POINT (-75.25763 39.94953),POINT (-5714985.845 8479845.686),"POLYGON ((-5715014.449 8479809.842, -5715014.8..."
470,735,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27542 39.96896, -75.27542 39.968...","POLYGON ((-75.27542 39.96896, -75.27541 39.968...",POINT (-75.27612 39.96913),POINT (-5712590.186 8483121.334),"POLYGON ((-5712589.472 8483026.846, -5712589.5..."
471,736,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...",242023,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,...,None,None,None,458.291779,1.188739e+04,"POLYGON ((-75.25664 39.94598, -75.25669 39.945...","POLYGON ((-75.25669 39.94597, -75.25671 39.945...",POINT (-75.25609 39.94665),POINT (-5715384.150 8479507.849),"POLYGON ((-5715513.534 8479540.584, -5715516.4..."
472,739,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...",242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,...,None,None,None,552.520121,1.592591e+04,"POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27789 39.96685, -75.27788 39.966...",POINT (-75.27714 39.96644),POINT (-5713059.443 8483108.295),"POLYGON ((-5713024.922 8483221.016, -5713022.5..."


In [11]:
start_time = time.time()
ParkRoad = pd.DataFrame()
mat = list()
# For all
for i in range(len(parks_within_range)):
    dist = road_nodes['geometry'].to_crs(3043).distance(parks_within_range['geometry'].to_crs(
        3043)[i])
    buf_nodes = road_nodes[(dist < 25) & (dist > 0)]
    mat.append(list(np.repeat(i, len(buf_nodes))))
    ParkRoad = pd.concat([ParkRoad, buf_nodes])
    if i % 10 == 0: print(round(i/len(parks_within_range)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')

# Park no list conversion
mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

# Format
ParkRoad['Park_No'] = mat_u
ParkRoad = ParkRoad.reset_index()
ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
print('100 % done',round((time.time() - start_time) / 60,2),' mns')
ParkRoad

    

0.0 % done 0.01  mns
2.1 % done 0.25  mns
4.2 % done 0.34  mns
6.3 % done 0.59  mns
8.4 % done 0.89  mns
10.5 % done 1.06  mns
12.7 % done 1.27  mns
14.8 % done 1.6  mns
16.9 % done 1.67  mns
19.0 % done 1.74  mns
21.1 % done 1.81  mns
23.2 % done 1.87  mns
25.3 % done 1.94  mns
27.4 % done 2.01  mns
29.5 % done 2.09  mns
31.6 % done 2.16  mns
33.8 % done 2.22  mns
35.9 % done 2.3  mns
38.0 % done 2.38  mns
40.1 % done 2.45  mns
42.2 % done 2.54  mns
44.3 % done 2.6  mns
46.4 % done 2.67  mns
48.5 % done 2.74  mns
50.6 % done 2.81  mns
52.7 % done 2.87  mns
54.9 % done 2.92  mns
57.0 % done 2.98  mns
59.1 % done 3.04  mns
61.2 % done 3.1  mns
63.3 % done 3.18  mns
65.4 % done 3.26  mns
67.5 % done 3.32  mns
69.6 % done 3.39  mns
71.7 % done 3.47  mns
73.8 % done 3.55  mns
75.9 % done 3.62  mns
78.1 % done 3.69  mns
80.2 % done 3.76  mns
82.3 % done 3.84  mns
84.4 % done 3.91  mns
86.5 % done 3.99  mns
88.6 % done 4.07  mns
90.7 % done 4.15  mns
92.8 % done 4.23  mns
94.9 % done 4.31  m

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m,Park_No,park_lon,park_lat
0,109991185,4.433758e+06,479471.157101,3,-75.240687,40.053816,None,None,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,8.483120e+06
1,109991191,4.433867e+06,479608.431472,3,-75.239081,40.054803,None,None,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,8.482975e+06
2,5550942021,4.433928e+06,479315.926381,3,-75.242512,40.055347,None,None,POINT (-75.24251 40.05535),POINT (-5697523.197 8483422.258),0,-5.697523e+06,8.483422e+06
3,109991199,4.433975e+06,479743.480201,3,-75.237501,40.055776,None,None,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,8.482831e+06
4,109991204,4.434025e+06,479804.853265,3,-75.236783,40.056232,None,None,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,8.482767e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,110166321,4.424071e+06,476384.161037,1,-75.276527,39.966462,None,None,POINT (-75.27653 39.96646),POINT (-5713031.271 8483034.574),472,-5.713031e+06,8.483035e+06
7342,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),473,-5.713613e+06,8.482370e+06
7343,110133414,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),473,-5.713713e+06,8.482492e+06
7344,923789083,4.423655e+06,476581.963819,3,-75.274196,39.962719,None,None,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),473,-5.713536e+06,8.482556e+06


In [180]:
# Save as file
ParkRoad[['Park_No','geometry']].to_crs(4326).to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

In [13]:
ParkRoad[ParkRoad['Park_No'] == 453]

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m,Park_No,park_lon,park_lat
7224,109890052,4.429296e+06,493169.700669,3,-75.080034,40.013838,None,None,POINT (-75.08003 40.01384),POINT (-5697684.279 8461406.828),453,-5.697684e+06,8.461407e+06
7225,110188472,4.429317e+06,493092.570321,3,-75.080938,40.014029,None,None,POINT (-75.08094 40.01403),POINT (-5697690.013 8461527.279),453,-5.697690e+06,8.461527e+06
7226,110291816,4.429273e+06,493150.989575,4,-75.080253,40.013627,None,None,POINT (-75.08025 40.01363),POINT (-5697726.578 8461422.644),453,-5.697727e+06,8.461423e+06


In [120]:
PtR = pd.merge(ParkRoad, parks_within_range[['geometry','geometry_w_buffer']], left_on = 'Park_No', right_index = True)
PtR['park_size_walkable'] = PtR['geometry_m'].buffer(500).to_crs(4326).intersection(PtR['geometry_y'])
PtR['walk_area'] = PtR['park_size_walkable'].to_crs(3043).area
PtR['park_area'] = PtR['geometry_y'].to_crs(3043).area
PtR['share_walked'] = PtR['walk_area'] / PtR['park_area']
PtR['size_infl_factor'] = PtR['walk_area'] / PtR['walk_area'].mean()
PtR

,osmid,y,x,street_cou,lon,lat,highway,ref,geometry_x,geometry_m,Park_No,park_lon,park_lat,geometry_y,geometry_w_buffer,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor
0,109991185,4.433758e+06,479471.157101,3,-75.240687,40.053816,None,None,POINT (-75.24069 40.05382),POINT (-5697695.034 8483120.448),0,-5.697695e+06,8.483120e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.23914 40.05656, -75.23880 40.056...",198723.068185,4.192678e+06,0.047398,1.504099
1,109991191,4.433867e+06,479608.431472,3,-75.239081,40.054803,None,None,POINT (-75.23908 40.05480),POINT (-5697474.612 8482974.522),0,-5.697475e+06,8.482975e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24278 40.05572, -75.24265 40.055...",291442.444263,4.192678e+06,0.069512,2.205875
2,5550942021,4.433928e+06,479315.926381,3,-75.242512,40.055347,None,None,POINT (-75.24251 40.05535),POINT (-5697523.197 8483422.258),0,-5.697523e+06,8.483422e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","MULTIPOLYGON (((-75.24062 40.05796, -75.24030 ...",440724.683283,4.192678e+06,0.105118,3.335766
3,109991199,4.433975e+06,479743.480201,3,-75.237501,40.055776,None,None,POINT (-75.23750 40.05578),POINT (-5697257.444 8482831.074),0,-5.697257e+06,8.482831e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24077 40.05416, -75.24096 40.054...",245064.061025,4.192678e+06,0.058450,1.854846
4,109991204,4.434025e+06,479804.853265,3,-75.236783,40.056232,None,None,POINT (-75.23678 40.05623),POINT (-5697156.554 8482766.601),0,-5.697157e+06,8.482767e+06,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...","POLYGON ((-75.25609 40.06118, -75.25608 40.061...","POLYGON ((-75.24005 40.05461, -75.24024 40.054...",187511.352028,4.192678e+06,0.044724,1.419240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,110166321,4.424071e+06,476384.161037,1,-75.276527,39.966462,None,None,POINT (-75.27653 39.96646),POINT (-5713031.271 8483034.574),472,-5.713031e+06,8.483035e+06,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...","POLYGON ((-75.27789 39.96684, -75.27789 39.966...","POLYGON ((-75.27647 39.96684, -75.27650 39.966...",19734.719624,1.973472e+04,1.000000,0.149369
7342,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-75.27300 39.96194),POINT (-5713613.274 8482370.035),473,-5.713613e+06,8.482370e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069
7343,110133414,4.423531e+06,476586.279558,3,-75.274141,39.961600,None,None,POINT (-75.27414 39.96160),POINT (-5713712.736 8482492.281),473,-5.713713e+06,8.482492e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069
7344,923789083,4.423655e+06,476581.963819,3,-75.274196,39.962719,None,None,POINT (-75.27420 39.96272),POINT (-5713536.406 8482556.428),473,-5.713536e+06,8.482556e+06,"POLYGON ((-75.27342 39.96180, -75.27493 39.961...","POLYGON ((-75.27332 39.96168, -75.27333 39.961...","POLYGON ((-75.27493 39.96162, -75.27515 39.961...",47176.323920,4.717632e+04,1.000000,0.357069


In [121]:
((gpd.GeoDataFrame(PtR['park_size_walkable'], geometry = 'park_size_walkable', crs = 4326).to_crs(3043).area.apply(
    math.log))-5).median()

6.511375567665073

In [122]:
ParkRoad['geometry']

0       POINT (-75.24069 40.05382)
1       POINT (-75.23908 40.05480)
2       POINT (-75.24251 40.05535)
3       POINT (-75.23750 40.05578)
4       POINT (-75.23678 40.05623)
                   ...            
7341    POINT (-75.27653 39.96646)
7342    POINT (-75.27300 39.96194)
7343    POINT (-75.27414 39.96160)
7344    POINT (-75.27420 39.96272)
7345    POINT (-75.27472 39.96248)
Name: geometry, Length: 7346, dtype: geometry

In [123]:
gpd.GeoDataFrame(PtR[['Park_No','park_size_walkable']], geometry = 'park_size_walkable', crs = 4326
                ).to_file(r'C:\Users\bartb\Downloads\parks.shp')

In [155]:
# Check all parks within 1000m radius
start_time = time.time()
len1 = len(popgrid)
len2 = len(ParkRoad)
len3 = int(np.ceil(len(ParkRoad)/1000))
output = pd.DataFrame()
len_mat = 0
# Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
for i in range(len3):
    # Check all grid-park combinations per 1000
    l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
    listed = pd.DataFrame(list(product(l1, l2)))
    
    # Merge grid and park information
    grid_merged = pd.merge(listed, 
                           popgrid[['grid_id','centroid','centroid_m','grid_lon','grid_lat']],
                           left_on = 0, right_index = True)
    node_merged = pd.merge(grid_merged, 
                           PtR[['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                               'size_infl_factor','share_walked','park_area','walk_area']], 
                           left_on = 1, right_index = True)
    
    # Preset index for merging
    len4 = len(node_merged)
    node_merged['key'] = range(0,len(node_merged))
    node_merged = node_merged.set_index('key')
    node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]
        
    # Create lists for better computational performance
    glon = list(node_merged['grid_lon'])
    glat = list(node_merged['grid_lat'])
    plon = list(node_merged['park_lon'])
    plat = list(node_merged['park_lat'])
    infl = list(node_merged['size_infl_factor'])
    
    # Get the euclidean distances
    mat = np.repeat(None,len(node_merged))
    mat2 = np.repeat(None,len(node_merged))
    for j in range(len(node_merged)):
        mat[j] = math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl[j]
        mat2[j] = math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2)
    
    # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
    mat_df = pd.DataFrame(mat)
    mat_df = mat_df[mat_df[0] <=1000].dropna()
    mat_df = pd.DataFrame(mat_df)
    mat_df = pd.merge(mat_df, pd.DataFrame(mat2), left_index = True, right_index = True, how = 'left')
    mat_df.columns = ['adjusted euclidean','raw euclidean']    
    mat_df = mat_df.join(node_merged)
        
    output = pd.concat([output, mat_df])

    print((i+1),'/',len3,'done',round((time.time() - start_time) / 60,2),' mns')
    print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within Gravity adjusted 1000m:',len(mat_df))
    
    # Checks the number of the parks within 1000m.
    len_mat = len_mat + len(mat_df)
    
# Renaming columns
print('total combinations within distance',len_mat)
output.columns = ['adjusted euclidean','raw euclidean','Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'grid_lon','grid_lat',
                  'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                  'park_lon','park_lat','size_infl_factor','parkshare_walked','park_area','walk_area']

output = output[['raw euclidean','size_infl_factor','adjusted euclidean','Grid_No','Park_entry_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','walk_area']]
output

1 / 8 done 0.23  mns
of 3891000 within Gravity adjusted 1000m: 51697
2 / 8 done 0.47  mns
of 3891000 within Gravity adjusted 1000m: 29405
3 / 8 done 0.7  mns
of 3891000 within Gravity adjusted 1000m: 54621
4 / 8 done 0.93  mns
of 3891000 within Gravity adjusted 1000m: 43334
5 / 8 done 1.16  mns
of 3891000 within Gravity adjusted 1000m: 5143
6 / 8 done 1.4  mns
of 3891000 within Gravity adjusted 1000m: 6131
7 / 8 done 1.63  mns
of 3891000 within Gravity adjusted 1000m: 5783
8 / 8 done 1.75  mns
of 346000 within Gravity adjusted 1000m: 1190
total combinations within distance 197304


,raw euclidean,size_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area
868,1373.392587,1.504099,913.099756,868,0,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
913,1089.848567,1.504099,724.585577,913,0,6264,POINT (-75.24247 40.06019),POINT (-5696750.675 8483664.465),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
914,1067.304885,1.504099,709.597415,914,0,6265,POINT (-75.24022 40.06019),POINT (-5696662.252 8483389.707),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
915,1121.233555,1.504099,745.45188,915,0,6266,POINT (-75.23797 40.06019),POINT (-5696573.814 8483114.956),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
916,1241.70838,1.504099,825.54954,916,0,6267,POINT (-75.23573 40.06019),POINT (-5696485.362 8482840.213),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168722,2215.270428,2.677589,827.337825,1422,7300,7685,POINT (-75.24022 40.04293),POINT (-5699410.543 8482505.011),463,5678234422,POINT (-75.25738 40.04175),"POLYGON ((-75.25235 40.04169, -75.25188 40.041...",POINT (-5700274.288 8484544.954),353765.666039
1168777,1919.873008,2.677589,717.015647,1477,7300,7826,POINT (-75.24247 40.04120),POINT (-5699773.963 8482691.420),463,5678234422,POINT (-75.25738 40.04175),"POLYGON ((-75.25235 40.04169, -75.25188 40.041...",POINT (-5700274.288 8484544.954),353765.666039
1168778,2208.40456,2.677589,824.773627,1478,7300,7827,POINT (-75.24022 40.04120),POINT (-5699685.454 8482416.498),463,5678234422,POINT (-75.25738 40.04175),"POLYGON ((-75.25235 40.04169, -75.25188 40.041...",POINT (-5700274.288 8484544.954),353765.666039
1168833,1955.075376,2.677589,730.162688,1533,7300,7968,POINT (-75.24247 40.03948),POINT (-5700048.897 8482602.914),463,5678234422,POINT (-75.25738 40.04175),"POLYGON ((-75.25235 40.04169, -75.25188 40.041...",POINT (-5700274.288 8484544.954),353765.666039


In [ ]:
park_entry_centroid = gpd.GeoDataFrame(output[['adjusted euclidean','Grid_No','Park_No','Parkroad_osmid',
                                               'Parkroad_coords_centroid']],geometry = 'Parkroad_coords_centroid', crs = 4326)
park_entry_centroid.to_file(r'C:\Users\bartb\Downloads\park_centroid.shp')

In [157]:
output[(output['Park_No'] == 231) & (output['Grid_No'] == 2639)]

,raw euclidean,size_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area
434540,214.025883,0.444812,481.159902,2639,5111,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809486,POINT (-75.11806 39.99394),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702367.869 8465026.271),58769.051358
438431,208.087815,0.444812,467.810302,2639,5112,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,5547581605,POINT (-75.11819 39.99395),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702370.518 8465042.292),58769.051358
442322,315.122553,0.444812,708.439254,2639,5113,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109809494,POINT (-75.11790 39.99456),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702262.806 8465038.940),58769.051358
450104,334.672172,0.444812,752.389512,2639,5115,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556737,POINT (-75.11788 39.99468),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702242.736 8465042.228),58769.051358
457886,257.779648,0.444812,579.524443,2639,5117,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,109999351,POINT (-75.11999 39.99417),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702407.718 8465274.162),58769.051358
461777,202.093837,0.444812,454.335006,2639,5118,11857,POINT (-75.11895 39.99285),POINT (-5702575.446 8465078.414),231,2371556736,POINT (-75.11909 39.99406),"POLYGON ((-75.11817 39.99397, -75.11989 39.994...",POINT (-5702389.554 8465157.699),58769.051358


In [158]:
pd.DataFrame(output[(output['Park_No'] == 231) & (output['Grid_No'] == 2500)])


,raw euclidean,size_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area


In [159]:
start_time = time.time()

# Reinstate geographic elements
gp_entry = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
gp_entry['Grid_m_centroid'] = gpd.GeoSeries(gp_entry['Grid_m_centroid'], crs = 3043)
gp_entry['Parkroad_coords_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_coords_centroid'], crs = 4326)
gp_entry['Parkroad_m_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_m_centroid'], crs = 3043)

# Get index for merging later
roads = road_nodes.reset_index()

# Get the nearest entrance point for the grid centroids
mat2 = np.repeat(None,len(gp_entry))
for i in range(len(gp_entry)):
    nearest = int(roads['geometry'].sindex.nearest(gp_entry['Grid_coords_centroid'].iloc[i])[1])
    mat2[i] = roads['osmid'].iloc[nearest]
    if i % 100000 == 0: print(round(i/len(gp_entry)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')
print(len(mat2[mat2 != None]))

# Add all the information and the distance from the nearest point to the grid centroid in a geodataframe
pd.DataFrame(mat2)
gp_entry['grid_osm'] = mat2
gp_entry = pd.merge(gp_entry, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry['geometry_m'] = gp_entry['geometry'].to_crs(3043)
gp_entry['grid_entry_dist'] = round(gp_entry['Grid_m_centroid'].distance(gp_entry['geometry_m'].to_crs(3043)),3)
gp_entry = gp_entry.reset_index()
print('100 % done', round((time.time() - start_time) / 60,2),' mns')
gp_entry

0.0 % done 0.0  mns
50.7 % done 0.16  mns
197304
100 % done 0.32  mns


,index,raw euclidean,size_infl_factor,adjusted euclidean,Grid_No,Park_entry_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Park_No,Parkroad_osmid,Park_geom,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area,grid_osm,geometry,geometry_m,grid_entry_dist
0,868,1373.392587,1.504099,913.099756,868,0,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,109991185,POINT (-75.24069 40.05382),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697695.034 8483120.448),198723.068185,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
1,4759,1266.184486,2.205875,574.005428,868,1,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,109991191,POINT (-75.23908 40.05480),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697474.612 8482974.522),291442.444263,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
2,8650,1098.225747,3.335766,329.227465,868,2,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,5550942021,POINT (-75.24251 40.05535),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697523.197 8483422.258),440724.683283,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
3,12541,1208.512478,1.854846,651.543362,868,3,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,109991199,POINT (-75.23750 40.05578),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697257.444 8482831.074),245064.061025,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
4,16432,1198.333947,1.419240,844.349224,868,4,6122,POINT (-75.24247 40.06191),POINT (-5696475.919 8483752.877),0,109991204,POINT (-75.23678 40.05623),"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",POINT (-5697156.554 8482766.601),187511.352028,110562147,POINT (-75.24257 40.06216),POINT (-5696439.791 8483778.131),44.080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197299,824565,264.127891,0.300130,880.045641,3564,7211,16657,POINT (-75.18183 39.93410),POINT (-5714439.752 8469744.047),452,110048580,POINT (-75.18239 39.93561),"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714220.271 8469890.988),39653.440881,110255941,POINT (-75.18106 39.93381),POINT (-5714454.475 8469634.992),110.045
197300,828456,209.658054,0.300130,698.558021,3564,7212,16657,POINT (-75.18183 39.93410),POINT (-5714439.752 8469744.047),452,110048592,POINT (-75.18247 39.93525),"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714281.714 8469881.817),39653.440881,110255941,POINT (-75.18106 39.93381),POINT (-5714454.475 8469634.992),110.045
197301,832347,162.167947,0.300130,540.326107,3564,7213,16657,POINT (-75.18183 39.93410),POINT (-5714439.752 8469744.047),452,9454221997,POINT (-75.18255 39.93489),"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714341.775 8469873.272),39653.440881,110255941,POINT (-75.18106 39.93381),POINT (-5714454.475 8469634.992),110.045
197302,836238,154.127823,0.300130,513.537282,3564,7214,16657,POINT (-75.18183 39.93410),POINT (-5714439.752 8469744.047),452,110048601,POINT (-75.18256 39.93482),"POLYGON ((-75.18408 39.93506, -75.18410 39.935...",POINT (-5714353.481 8469871.768),39653.440881,110255941,POINT (-75.18106 39.93381),POINT (-5714454.475 8469634.992),110.045


In [128]:
# Get only necessary road connections columns for network performance
road_conn = road_connections[['osmid','key','length','geometry']]
road_conn = road_conn.set_index('key')

In [129]:
#
gp_entry['Grid_m_centroid'].distance(gp_entry['Parkroad_m_centroid'].to_crs(3043))

0         1373.392587
1         1266.184486
2         1098.225747
3         1208.512478
4         1198.333947
             ...     
197299     264.127891
197300     209.658054
197301     162.167947
197302     154.127823
197303     159.786183
Length: 197304, dtype: float64

In [160]:
import warnings
warnings.filterwarnings("ignore")

start_time = time.time()

parknode = list(gp_entry['Parkroad_osmid'])
gridnode = list(gp_entry['grid_osm'])

s_mat = list([])
s_mat1 = list([])
s_mat2 = list([])
s_mat3 = list([])
s_mat4 = list([])

for i in range(len(gp_entry)):
    # First try from population grid to park Dijkstra algorithm
    try:
        shortest = nx.shortest_path(graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
        s_mat.append(shortest)
        shortest_to = list(shortest[1:len(shortest)])
        shortest_to.append(0)
        s_mat1.append(shortest_to)
        s_mat2.append(list(np.repeat(i, len(shortest))))
        s_mat3.append(list(np.arange(0, len(shortest))))
        s_mat4.append('normal way')
        
    except:
        try:
            # Check the reverse
            shortest = nx.shortest_path(graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
            s_mat.append(shortest)
            shortest_to = list(shortest[1:len(shortest)])
            shortest_to.append(0)
            s_mat1.append(shortest_to)
            s_mat2.append(list(np.repeat(i, len(shortest))))
            s_mat3.append(list(np.arange(0, len(shortest))))
            s_mat4.append('reverse way')
        except:
            try:
                # Get the route to the nearest park node
                new_node = pd.DataFrame((abs(gp_entry['Parkroad_coords_centroid'].x[123381] - PtR['geometry_y'].x)**2
                                         + abs(gp_entry['Parkroad_coords_centroid'].y[123381] - PtR['geometry_y'].y)**2
                                        )**(1/2)).join(PtR['osmid']).sort_values(0).iloc[1,1]
                shortest = nx.shortest_path(graph, gridnode[i], new_node, 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('nearest way')
            except:
                # Print if none of the above three options gives any result
                print('index',i,'no route possible between osmid',gridnode[i],'and',parknode[i])
                pass
            
    if i % 10000 == 0: print(round(i/len(gp_entry)*100,2),'% done', round((time.time() - start_time) / 60,2),' mns')
print('100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')


0.0 % done 0.0  mns
5.07 % done 0.23  mns
10.14 % done 0.69  mns
15.2 % done 1.51  mns
20.27 % done 2.15  mns
25.34 % done 2.46  mns
30.41 % done 2.76  mns
35.48 % done 3.27  mns
40.55 % done 3.89  mns
45.61 % done 4.3  mns
50.68 % done 4.68  mns
55.75 % done 5.47  mns
60.82 % done 5.84  mns
65.89 % done 6.54  mns
70.96 % done 6.96  mns
76.02 % done 7.47  mns
81.09 % done 8.01  mns
86.16 % done 8.81  mns
91.23 % done 10.02  mns
96.3 % done 10.29  mns
100 % pathfinding done 10.49  mns


In [161]:
start_time = time.time()
# Unpack lists and add them in a dataframe
s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()

# Format df
routes.columns = ['from','to','route','step']
mat_key = list([])
for i in range(len(routes)):
    mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
routes['key'] = mat_key
routes = routes.set_index('key')

# Add route information
routes = routes.join(road_conn, how = 'left')
routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
routes.sort_values(by = ['route','step'])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes

time taken 1.72 mns


,from,to,route,step,osmid,length,geometry
key,,,,,,,
1015355888-0,1015355888,0,23974,23,NaN,NaN,None
1015355888-0,1015355888,0,24137,22,NaN,NaN,None
1015355888-0,1015355888,0,24353,27,NaN,NaN,None
1015355888-0,1015355888,0,24697,24,NaN,NaN,None
1015355888-0,1015355888,0,24891,20,NaN,NaN,None
...,...,...,...,...,...,...,...
983494521-3663519582,983494521,3663519582,30068,19,"[700438096, 498410505]",47.77,"LINESTRING (-75.13963 39.94622, -75.13966 39.9..."
983494521-3663519582,983494521,3663519582,30322,15,"[700438096, 498410505]",47.77,"LINESTRING (-75.13963 39.94622, -75.13966 39.9..."
983494521-3663519582,983494521,3663519582,30566,17,"[700438096, 498410505]",47.77,"LINESTRING (-75.13963 39.94622, -75.13966 39.9..."


In [162]:
start_time = time.time()
routegrid = list([])
for i in range(len(gp_entry)):
    routegrid.append(LineString([(gp_entry['geometry'][i]),(gp_entry['Grid_coords_centroid'][i])]))
gridline = gpd.GeoDataFrame(geometry = routegrid, crs = 4326)
gridline = gridline.join(gp_entry[['grid_entry_dist']])
gridline = gridline.reset_index()
gridline.columns = ['route','geometry','length']
gridline['grid_osm'] = '0-' + gp_entry['grid_osm'].astype(str)
gridline = gridline.set_index('grid_osm')
gridline['step'] = -1
gridline['osmid'] = np.nan
gridline['to'] = np.where(s_mat4 == 'normal way',gp_entry['Parkroad_osmid'],gp_entry['grid_osm'])

gridline['from'] = 0
gridline = gridline[['from','to','route','step','osmid','length','geometry']]
routes_stack = pd.concat([routes, gridline])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes_stack[routes_stack['route'] == 1].sort_values(by = 'step')

time taken 0.43 mns


,from,to,route,step,osmid,length,geometry
0-110562147,0,110562147,1,-1,NaN,44.080,"LINESTRING (-75.24257 40.06216, -75.24247 40.0..."
110562147-110546460,110562147,110546460,1,0,12205057,70.766,"LINESTRING (-75.24257 40.06216, -75.24294 40.0..."
110546460-110422537,110546460,110422537,1,1,"[12205057, 12182622]",206.378,"LINESTRING (-75.24499 40.06143, -75.24486 40.0..."
110422537-110422523,110422537,110422523,1,2,12182622,153.720,"LINESTRING (-75.24499 40.06143, -75.24566 40.0..."
110422523-110430427,110422523,110430427,1,3,12184719,50.287,"LINESTRING (-75.24687 40.06192, -75.24643 40.0..."
110430427-2281528216,110430427,2281528216,1,4,12187778,198.727,"LINESTRING (-75.24529 40.06060, -75.24645 40.0..."
2281528216-4774911885,2281528216,4774911885,1,5,12187778,137.150,"LINESTRING (-75.24418 40.05970, -75.24529 40.0..."
4774911885-4244000117,4774911885,4244000117,1,6,12187778,102.832,"LINESTRING (-75.24418 40.05970, -75.24343 40.0..."
4244000117-110443757,4244000117,110443757,1,7,12187778,232.152,"LINESTRING (-75.24336 40.05903, -75.24160 40.0..."
110443757-110305724,110443757,110305724,1,8,108593573,226.652,"LINESTRING (-75.24148 40.05751, -75.24177 40.0..."


In [169]:
start_time = time.time()
# get single (dissolved) line per route, attach information.
routes2 = routes_stack[['route','geometry']].dissolve('route')
routes2['way_calculated'] = s_mat4
routes2['route_cost'] = routes.groupby('route')['length'].sum()
routes2['num_steps'] = routes.groupby('route')['step'].max()
routes2.index = routes2.index.astype(int)
routes2 = pd.merge(routes2, gp_entry[['Grid_No','Park_No',
                                      'Park_entry_No','grid_entry_dist','Parkroad_osmid','grid_osm','walk_area','size_infl_factor']],
                   left_index = True, right_index = True)
routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
routes2['grav_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_factor']
routes2['parkcost_km2'] = routes2['grav_total_cost'] * (routes2['walk_area'] /1000000)
routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes2 

time taken 0.91 mns


,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,walk_area,size_infl_factor,raw_total_cost,grav_total_cost,parkcost_km2,gridpark_no
route,,,,,,,,,,,,,,,,
0,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,1726.207,11,868,0,0,44.080,109991185,110562147,198723.068185,1.504099,1770.287,1176.974918,233.892067,868-0
1,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,1901.505,12,868,0,1,44.080,109991191,110562147,291442.444263,2.205875,1945.585,882.001290,257.052612,868-0
2,"MULTILINESTRING ((-75.24344 40.05613, -75.2426...",normal way,1495.763,10,868,0,2,44.080,5550942021,110562147,440724.683283,3.335766,1539.843,461.616029,203.445578,868-0
3,"MULTILINESTRING ((-75.23908 40.05480, -75.2406...",normal way,2074.101,13,868,0,3,44.080,109991199,110562147,245064.061025,1.854846,2118.181,1141.971469,279.856166,868-0
4,"MULTILINESTRING ((-75.23938 40.06072, -75.2397...",normal way,1456.034,13,868,0,4,44.080,109991204,110562147,187511.352028,1.419240,1500.114,1056.984237,198.196543,868-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197299,"MULTILINESTRING ((-75.18072 39.93540, -75.1808...",normal way,322.953,5,3564,452,7211,110.045,110048580,110255941,39653.440881,0.300130,432.998,1442.702629,57.208123,3564-452
197300,"MULTILINESTRING ((-75.18239 39.93561, -75.1824...",normal way,364.164,6,3564,452,7212,110.045,110048592,110255941,39653.440881,0.300130,474.209,1580.013236,62.652961,3564-452
197301,"MULTILINESTRING ((-75.18256 39.93482, -75.1826...",normal way,251.812,6,3564,452,7213,110.045,9454221997,110255941,39653.440881,0.300130,361.857,1205.668491,47.808904,3564-452


In [171]:
start_time = time.time()
# Get the entry point of the park that is closest to the grid centroid
gridpark_score = routes2.iloc[routes2.groupby('gridpark_no')['grav_total_cost'].idxmin()]

# Check if the route to the closest entry point is within 1000m.
gridpark_score = gridpark_score[gridpark_score['grav_total_cost'] <= 1000]

# The score is the inverse of the total cost (if a park is 100m (on route) away its score is 900 (1000 - 100))
gridpark_score['score'] = 1000 - gridpark_score['grav_total_cost']

# Get the population weighted score
gridpark_score = pd.merge(gridpark_score, popgrid[['PoP2015_Number','geometry']],
                          left_on = 'Grid_No', right_index = True, how = 'outer')

gridpark_score['score'] = gridpark_score['score'].fillna(0).round()
gridpark_score['population_score'] = gridpark_score['score'] * gridpark_score['PoP2015_Number']

gridpark_score['Park_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score['Park_entry_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_entry_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score.index = gridpark_score.index.fillna(-1)
gridpark_score.index = gridpark_score.index.astype(int)

gridpark_score['num_steps'] = gridpark_score['num_steps'].fillna(-1)
gridpark_score['num_steps'] = gridpark_score['num_steps'].astype(int)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')

gridpark_score

time taken 0.01 mns


,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,Park_entry_No,grid_entry_dist,Parkroad_osmid,grid_osm,walk_area,size_infl_factor,raw_total_cost,grav_total_cost,parkcost_km2,gridpark_no,score,PoP2015_Number,geometry_y,population_score
136044,"MULTILINESTRING ((-74.99039 40.05713, -74.9913...",normal way,1027.223,13,1000,35,35,20.765,1.099025e+08,7826688686,311959.970992,2.361169,1047.988,443.842872,138.461210,1000-35,556.0,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",103972.0
136064,"LINESTRING (-74.99992 40.05834, -74.99992 40.0...",normal way,0.000,0,1000,45,45,20.765,7.826689e+09,7826688686,263123.707902,1.991536,20.765,10.426625,2.743492,1000-45,990.0,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",185130.0
136085,"MULTILINESTRING ((-74.99333 40.05669, -74.9945...",normal way,415.485,4,1001,35,35,58.594,1.104250e+08,110425022,281553.844925,2.131030,474.079,222.464679,62.635786,1001-35,778.0,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",44346.0
136109,"LINESTRING (-74.99565 40.05877, -74.99543 40.0...",normal way,0.000,0,1001,45,45,58.594,1.104250e+08,110425022,199688.560770,1.511407,58.594,38.767855,7.741497,1001-45,961.0,57,"POLYGON ((-74.99431 40.05932, -74.99431 40.057...",54777.0
136124,"MULTILINESTRING ((-74.98938 40.05751, -74.9905...",normal way,234.701,2,1002,35,35,52.428,1.099025e+08,110013037,311959.970992,2.361169,287.129,121.604599,37.935767,1002-35,878.0,45,"POLYGON ((-74.98981 40.05932, -74.98981 40.057...",39510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-1,None,NaN,NaN,-1,3885,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5,"POLYGON ((-75.25033 39.88481, -75.25033 39.883...",0.0
-1,None,NaN,NaN,-1,3887,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,39,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",0.0
-1,None,NaN,NaN,-1,3888,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",0.0
-1,None,NaN,NaN,-1,3889,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",0.0


In [140]:
gridpark_lines = gpd.GeoDataFrame(gridpark_score, geometry = 'geometry_x', crs = 4326)
gridpark_nodes = gpd.GeoDataFrame(gridpark_score, geometry = 'geometry_y', crs = 4326)

gridpark_lines[['geometry_x','grav_total_cost','Grid_No','Park_No','PoP2015_Number','population_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_lines.shp')

gridpark_nodes[['geometry_y','grav_total_cost','Grid_No','Park_No','PoP2015_Number','population_score']].to_file(
    r'C:\Users\bartb\Downloads\gridpark_nodes.shp')

In [173]:
start_time = time.time()
# Group the park scores per grid.
grid_score = pd.DataFrame(gridpark_score.groupby('Grid_No')['score'].sum().sort_values(ascending=False))
grid_score['population_score (mln)'] = gridpark_score.groupby('Grid_No')['population_score'].sum()/1000000
grid_score['parkscore_km2'] = gridpark_score.groupby('Grid_No')['parkcost_km2'].sum()
grid_score['count'] = gridpark_score.groupby('Grid_No')['score'].count()

grid_score = grid_score.join(popgrid[['geometry','PoP2015_Number']])
grid_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Grid_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

# The merge contains grids with no access to parks within 1000m, these scores are set to 0
grid_score = gpd.GeoDataFrame(grid_score, geometry = 'geometry', crs = 4326)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
grid_score

time taken 0.02 mns


,score,population_score (mln),parkscore_km2,count,geometry,PoP2015_Number,lines
Grid_No,,,,,,,
911,2717.0,0.567853,354.008806,5,"POLYGON ((-74.98757 40.06277, -74.98757 40.061...",209,"(LINESTRING (-74.988392 40.06164749999999, -74..."
1240,2705.0,0.008115,680.141933,5,"POLYGON ((-75.00104 40.05069, -75.00104 40.048...",3,"(LINESTRING (-74.9971166 40.0479433, -74.99733..."
2221,2614.0,0.206506,250.593745,4,"POLYGON ((-75.20092 40.01444, -75.20092 40.012...",79,"(LINESTRING (-75.1990873 40.014659099999996, -..."
1047,2359.0,0.526057,597.834389,4,"POLYGON ((-75.01003 40.05760, -75.01003 40.055...",223,(LINESTRING (-75.02571000000002 40.05639809999...
1659,2302.0,0.246314,888.157576,4,"POLYGON ((-75.17397 40.03689, -75.17397 40.035...",107,"(LINESTRING (-75.1752609 40.0363294, -75.17514..."
...,...,...,...,...,...,...,...
2732,0.0,0.000000,0.000000,1,"POLYGON ((-75.14253 39.98853, -75.14253 39.986...",499,None
2733,0.0,0.000000,0.000000,1,"POLYGON ((-75.14028 39.98853, -75.14028 39.986...",515,None
2734,0.0,0.000000,0.000000,1,"POLYGON ((-75.13579 39.98853, -75.13579 39.986...",355,None


In [178]:
start_time = time.time()
# group the grid scores per park (which parks will be popular)
park_score = pd.DataFrame(gridpark_score.groupby('Park_No')['score'].sum().sort_values(ascending=False))
park_score['population_score (mln.)'] = gridpark_score.groupby('Park_No')['population_score'].sum()/1000000
park_score['parkscore_km2'] = gridpark_score.groupby('Park_No')['parkcost_km2'].sum()
park_score['Park access to pop.'] = gridpark_score.groupby('Park_No')['PoP2015_Number'].sum()
park_score['count'] = gridpark_score.groupby('Park_No')['score'].count()
park_score = park_score.join(parks_within_range[['geometry','OBJECTID']], how = 'outer')
park_score['park_area_m2'] = gpd.GeoSeries(park_score['geometry'], crs = 4326).to_crs(3043).area
park_score['park_score_km2_vs_total_park_size'] = park_score['parkscore_km2'] / (park_score['park_area_m2'] / 1000000)
park_score = park_score[park_score.index >= 0]
park_score['OBJECTID'] = park_score['OBJECTID'].astype(int)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
park_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Park_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
park_score

time taken 0.0 mns


,score,population_score (mln.),parkscore_km2,Park access to pop.,count,geometry,OBJECTID,park_area_m2,park_score_km2_vs_total_park_size,lines
0,51605.0,5.814463,26361.279273,11941.0,105.0,"MULTIPOLYGON (((-75.25221 40.05012, -75.25296 ...",81288,4.192678e+06,6287.456583,"(LINESTRING (-75.2410148 40.0578267, -75.24021..."
1,6.0,0.001062,54.063644,177.0,1.0,"POLYGON ((-75.24702 39.91695, -75.24706 39.916...",82731,5.470200e+04,988.330231,"(LINESTRING (-75.2458592 39.91941099999998, -7..."
2,126974.0,60.041728,59478.733045,126199.0,259.0,"MULTIPOLYGON (((-75.25450 39.95036, -75.25451 ...",82731,7.283218e+06,8166.545522,"(LINESTRING (-75.2578459 39.973964599999995, -..."
3,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.14985 39.96195, -75.14993 39.961...",1260,4.811379e+03,NaN,None
4,NaN,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((-75.17212 40.03366, -75.17215 ...",1348,1.069010e+04,NaN,None
...,...,...,...,...,...,...,...,...,...,...
469,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.25746 39.94945, -75.25807 39.949...",242021,3.624449e+03,NaN,None
470,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.27556 39.96885, -75.27571 39.968...",242022,9.827071e+03,NaN,None
471,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.25665 39.94613, -75.25671 39.946...",242023,2.709308e+04,NaN,None
472,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-75.27769 39.96684, -75.27647 39.966...",242035,1.973472e+04,NaN,None


In [ ]:
park_lines = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'Park access to pop.', 'lines']], 
                              geometry = 'lines', crs = 4326)
park_lines['Park_No'] = park_lines.index
park_lines.to_file(r'C:\Users\bartb\Downloads\park_entrance_routes.shp')

park_s = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'Park access to pop.', 'geometry']], 
                          geometry = 'geometry', crs = 4326)
park_s['Park_No'] = park_lines.index
park_s.to_file(r'C:\Users\bartb\Downloads\park_score_entrance.shp')

In [ ]:
grid_lines = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','lines']], 
                              geometry = 'lines', crs = 4326)
grid_lines.to_file(r'C:\Users\bartb\Downloads\grid_entrance_routes.shp')

grid_s = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','geometry']], 
                          geometry = 'geometry', crs = 4326)
grid_s.to_file(r'C:\Users\bartb\Downloads\grid_score_entrance.shp')

In [185]:
print(round((time.time() - start) / 60,2),'mns')

25.66 mns
